In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [1]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings


In [2]:
# 비타민 별 섭취할 수 있는 음식 정보
doc_list_1 = [
    "비타민A : 당근, 시금치, 감자 등의 주황색과 녹색 채소에서 섭취할 수 있습니다.",
    "비타민B : 전곡물, 콩, 견과류, 육류 등 다양한 식품에서 찾을 수 있습니다.",
    "비타민C : 오렌지, 키위, 딸기, 브로콜리, 피망 등의 과일과 채소에 많이 들어 있습니다.",
    "비타민D : 연어, 참치, 버섯, 우유, 계란 노른자 등에 함유되어 있습니다.",
    "비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.",
]

# 비타민 별 효능 정보
doc_list_2 = [
    "비타민A : 시력과 피부 건강을 지원합니다.",
    "비타민B : 에너지 대사와 신경계 기능을 돕습니다.",
    "비타민C : 면역 체계를 강화하고 콜라겐 생성을 촉진합니다.",
    "비타민D : 뼈 건강과 면역 체계를 지원합니다.",
    "비타민E : 항산화 작용을 통해 세포를 보호합니다.",
]
# bm25 retriever와 faiss retriever를 초기화합니다.
bm25_retriever = BM25Retriever.from_texts(
    # doc_list_1의 텍스트와 메타데이터를 사용하여 BM25Retriever를 초기화합니다.
    doc_list_1,
    metadatas=[{"source": 1}] * len(doc_list_1),
)
bm25_retriever.k = 1  # BM25Retriever의 검색 결과 개수를 1로 설정합니다.

embedding = HuggingFaceBgeEmbeddings()  # HuggingfaceBge 임베딩을 사용합니다.
faiss_vectorstore = FAISS.from_texts(
    # doc_list_2의 텍스트와 임베딩, 메타데이터를 사용하여 FAISS 벡터 저장소를 초기화합니다.
    doc_list_2,
    embedding,
    metadatas=[{"source": 2}] * len(doc_list_2),
)
# FAISS 벡터 저장소를 사용하여 retriever를 생성하고, 검색 결과 개수를 1로 설정합니다.
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 1})

# 앙상블 retriever를 초기화합니다.
ensemble_retriever = EnsembleRetriever(
    # BM25Retriever와 FAISS retriever를 사용하여 EnsembleRetriever를 초기화하고, 각 retriever의 가중치를 설정합니다.
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.6, 0.4],
    search_type="mmr",
)

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
# 검색 결과 문서를 가져옵니다.
query = "비타민A 의 효능은?"
ensemble_result = ensemble_retriever.get_relevant_documents(query)
bm25_result = bm25_retriever.get_relevant_documents(query)
faiss_result = faiss_retriever.get_relevant_documents(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]\n", ensemble_result, end="\n\n")
print("[BM25 Retriever]\n", bm25_result, end="\n\n")
print("[FAISS Retriever]\n", faiss_result, end="\n\n")

[Ensemble Retriever]
 [Document(page_content='비타민A : 당근, 시금치, 감자 등의 주황색과 녹색 채소에서 섭취할 수 있습니다.', metadata={'source': 1}), Document(page_content='비타민A : 시력과 피부 건강을 지원합니다.', metadata={'source': 2})]

[BM25 Retriever]
 [Document(page_content='비타민A : 당근, 시금치, 감자 등의 주황색과 녹색 채소에서 섭취할 수 있습니다.', metadata={'source': 1})]

[FAISS Retriever]
 [Document(page_content='비타민A : 시력과 피부 건강을 지원합니다.', metadata={'source': 2})]



/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [4]:
from langchain_core.runnables import ConfigurableField

In [5]:
faiss_retriever = faiss_vectorstore.as_retriever(
    # 검색 시 반환할 결과의 개수를 설정합니다.
    search_kwargs={"k": 2}
).configurable_fields(
    search_kwargs=ConfigurableField(
        # 검색 매개변수의 고유 식별자를 설정합니다.
        id="search_kwargs_faiss",
        # 검색 매개변수의 이름을 설정합니다.
        name="Search Kwargs",
        # 검색 매개변수에 대한 설명을 작성합니다.
        description="The search kwargs to use",
    )
)

In [6]:
ensemble_retriever = EnsembleRetriever(
    # 리트리버 목록을 설정합니다. 여기서는 bm25_retriever와 faiss_retriever를 사용합니다.
    retrievers=[bm25_retriever, faiss_retriever],
    # 각 리트리버의 가중치를 설정합니다. 여기서는 두 리트리버에 동일한 가중치 0.5를 부여합니다.
    weights=[0.5, 0.5],
)

In [7]:
# 검색 설정을 지정합니다. Faiss 검색에서 k=1로 설정하여 가장 유사한 문서 1개를 반환하도록 합니다.
config = {"configurable": {"search_kwargs_faiss": {"k": 1}}}
# 앙상블 검색기를 사용하여 "apples"에 대한 검색을 수행하고, 설정된 config를 적용합니다. 검색 결과는 docs 변수에 저장됩니다.
docs = ensemble_retriever.invoke("apples", config=config)
docs  # 검색 결과인 docs를 출력합니다.

[Document(page_content='비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.', metadata={'source': 1}),
 Document(page_content='비타민C : 면역 체계를 강화하고 콜라겐 생성을 촉진합니다.', metadata={'source': 2})]